# Cleaning File
Get the basic company information here.

In [1]:
import pickle
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import geopandas as gpd

import numpy as np

In [3]:
file = "data/compustat_2010s.csv"
compustat = pd.read_csv(file, encoding = 'unicode_escape')

In [4]:
compustat = compustat[(compustat.fyearq > 2009) & (compustat.fyearq < 2020)].reset_index()[['fyearq', 'fqtr', 'gvkey', 'conm', 'tic', 'naics','curcdq',
           'niq', 'revtq', 'cogsq', 'invtq']]
compustat.head()

,fyearq,fqtr,gvkey,conm,tic,naics,curcdq,niq,revtq,cogsq,invtq
0,2010,1.0,1004,AAR CORP,AIR,423860.0,USD,13.674,404.393,334.392,520.001
1,2010,2.0,1004,AAR CORP,AIR,423860.0,USD,16.814,454.858,379.795,524.548
2,2010,3.0,1004,AAR CORP,AIR,423860.0,USD,17.918,458.035,379.242,523.464
3,2010,4.0,1004,AAR CORP,AIR,423860.0,USD,21.420,487.826,399.272,507.274
4,2011,1.0,1004,AAR CORP,AIR,423860.0,USD,16.600,485.500,394.000,548.929


In [5]:
compustat.rename(columns = {'chq': 'cash', 
                            'fyearq': 'year',
                            'fqtr': 'qtr',
                           'conm': 'companyName',
                           'niq': 'netIncome',
                           'revtq': 'totalRevenue',
                           'cogsq': 'costGoodsSold',
                            'invtq': 'totalInv'
                           },
                 inplace = True)

In [6]:
compustat.head()

,year,qtr,gvkey,companyName,tic,naics,curcdq,netIncome,totalRevenue,costGoodsSold,totalInv
0,2010,1.0,1004,AAR CORP,AIR,423860.0,USD,13.674,404.393,334.392,520.001
1,2010,2.0,1004,AAR CORP,AIR,423860.0,USD,16.814,454.858,379.795,524.548
2,2010,3.0,1004,AAR CORP,AIR,423860.0,USD,17.918,458.035,379.242,523.464
3,2010,4.0,1004,AAR CORP,AIR,423860.0,USD,21.420,487.826,399.272,507.274
4,2011,1.0,1004,AAR CORP,AIR,423860.0,USD,16.600,485.500,394.000,548.929


In [7]:
compustat.curcdq.value_counts()

USD    372903
CAD     83685
Name: curcdq, dtype: int64

In [21]:
compustatNext = compustat.copy()

compustatNext['year'] -= 1
compustatNext.rename(columns = {'netIncome':'netIncomeNext',
                               'totalRevenue':'totalRevenueNext',
                               'costGoodsSold':'costGoodsSoldNext',
                               'totalInv':'totalInvNext'
                           },
                 inplace = True)

In [22]:
compustatNext.head()

,year,qtr,gvkey,companyName,tic,naics,curcdq,netIncomeNext,totalRevenueNext,costGoodsSoldNext,totalInvNext
0,2009,1.0,1004,AAR CORP,AIR,423860.0,USD,13.674,404.393,334.392,520.001
1,2009,2.0,1004,AAR CORP,AIR,423860.0,USD,16.814,454.858,379.795,524.548
2,2009,3.0,1004,AAR CORP,AIR,423860.0,USD,17.918,458.035,379.242,523.464
3,2009,4.0,1004,AAR CORP,AIR,423860.0,USD,21.420,487.826,399.272,507.274
4,2010,1.0,1004,AAR CORP,AIR,423860.0,USD,16.600,485.500,394.000,548.929


In [23]:
compustatChanges = compustat.merge(compustatNext)

In [24]:
compustatChanges

,year,qtr,gvkey,companyName,tic,naics,curcdq,netIncome,totalRevenue,costGoodsSold,totalInv,netIncomeNext,totalRevenueNext,costGoodsSoldNext,totalInvNext
0,2010,1.0,1004,AAR CORP,AIR,423860.0,USD,13.674,404.393,334.392,520.001,16.600,485.500,394.000,548.929
1,2010,2.0,1004,AAR CORP,AIR,423860.0,USD,16.814,454.858,379.795,524.548,17.581,481.957,386.493,570.066
2,2010,3.0,1004,AAR CORP,AIR,423860.0,USD,17.918,458.035,379.242,523.464,20.663,534.195,428.337,612.071
3,2010,4.0,1004,AAR CORP,AIR,423860.0,USD,21.420,487.826,399.272,507.274,12.879,572.846,453.578,599.752
4,2011,1.0,1004,AAR CORP,AIR,423860.0,USD,16.600,485.500,394.000,548.929,18.200,550.500,434.000,606.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382293,2018,4.0,345980,CONTEXTLOGIC INC,WISH,454110.0,USD,-208.000,1728.000,270.000,0.000,-124.000,576.000,185.000,0.000
382294,2018,1.0,347085,KAROOOOO LTD,KARO,518210.0,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
382295,2018,2.0,347085,KAROOOOO LTD,KARO,518210.0,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
382296,2018,3.0,347085,KAROOOOO LTD,KARO,518210.0,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.620


In [18]:
(compustatChanges.netIncomeNext[0])# - compustatChanges.netIncome[0])/compustatChanges.netIncome[0]

13.674

In [25]:
compustatChanges['incomeChange'] = (compustatChanges.netIncomeNext - compustatChanges.netIncome)/compustatChanges.netIncome

compustatChanges['revenueChange'] = (compustatChanges.totalRevenueNext - compustatChanges.totalRevenue)/compustatChanges.totalRevenue

compustatChanges['costChange'] = (compustatChanges.costGoodsSoldNext - compustatChanges.costGoodsSold)/compustatChanges.costGoodsSold

compustatChanges['inventoryChange'] = (compustatChanges.totalInvNext - compustatChanges.totalInv)/compustatChanges.totalInv

compustatChanges = compustatChanges[['year', 'qtr', 'gvkey', 'companyName', 
                                     'tic', 'naics', 'curcdq',
                                    'incomeChange','revenueChange','costChange','inventoryChange']]


In [27]:
compustatChanges.to_csv("data/compustatChanges_2010s.csv")

In [26]:
compustatChanges.head()

,year,qtr,gvkey,companyName,tic,naics,curcdq,incomeChange,revenueChange,costChange,inventoryChange
0,2010,1.0,1004,AAR CORP,AIR,423860.0,USD,0.213983,0.200565,0.178258,0.055631
1,2010,2.0,1004,AAR CORP,AIR,423860.0,USD,0.045617,0.059577,0.017636,0.086776
2,2010,3.0,1004,AAR CORP,AIR,423860.0,USD,0.153198,0.166276,0.129456,0.169270
3,2010,4.0,1004,AAR CORP,AIR,423860.0,USD,-0.398739,0.174283,0.136013,0.182304
4,2011,1.0,1004,AAR CORP,AIR,423860.0,USD,0.096386,0.133883,0.101523,0.104150


---------------------------------------

# Stock price data

In [67]:
file = "data/crsp_raw.csv"
raw = pd.read_csv(file, encoding = 'unicode_escape')

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [68]:
raw.head()

,PERMNO,date,EXCHCD,SICCD,TICKER,COMNAM,NAICS,PRIMEXCH,PERMCO,PRC,VOL,RET,RETX
0,15580,20000103,3.0,6320.0,AAME,ATLANTIC AMERICAN CORP,NaN,Q,5,2.3125,5221.0,0.000000,0.000000
1,15580,20000104,3.0,6320.0,AAME,ATLANTIC AMERICAN CORP,NaN,Q,5,2.3750,100.0,0.027027,0.027027
2,15580,20000105,3.0,6320.0,AAME,ATLANTIC AMERICAN CORP,NaN,Q,5,2.4375,7985.0,0.026316,0.026316
3,15580,20000106,3.0,6320.0,AAME,ATLANTIC AMERICAN CORP,NaN,Q,5,2.3125,500.0,-0.051282,-0.051282
4,15580,20000107,3.0,6320.0,AAME,ATLANTIC AMERICAN CORP,NaN,Q,5,2.3750,4530.0,0.027027,0.027027


Let's follow Evan's crsp cleaning and linking process.

1. Drop firms not traded on NYSE or NASDAQ.

In [69]:
other_exch = crsp[ (raw['PRIMEXCH'] != 'Q') & (raw['PRIMEXCH'] != 'N') ].index
raw.drop(other_exch, inplace=True)
raw.shape

(22272108, 13)

2. Drop entries with missing return or volume information.

In [70]:
raw.dropna(subset=['VOL', 'RET','date'],inplace=True)
raw['VOL'] = raw['VOL'].astype('int')
raw.drop(raw[raw['RET']=='C'].index,inplace=True)
raw['RET'] = raw['RET'].astype('float')
raw['date'] = raw['date'].astype('int')

3. Drop tickers with average daily trading volume less than 100K shares.

In [71]:
adv = raw[['VOL','TICKER']].groupby(['TICKER']).mean()
low_vol = adv.drop(adv[adv['VOL']>=100000].index)
low_vol_index = raw[raw['TICKER'].isin(low_vol.index)].index
raw.drop(low_vol_index,inplace=True)
raw.shape

(15589271, 13)

4. Drop tickers that traded for less than 20% of the period

In [72]:
ndays = len(raw.date.unique())
obs = raw['TICKER'].value_counts()
rare_obs = obs[obs<0.2*ndays]
rare_index = raw[raw['TICKER'].isin(rare_obs.index)].index
raw.drop(rare_index,inplace=True)
raw.shape

(14471739, 13)

In [108]:
raw = raw.drop_duplicates()

In [109]:
raw.shape

(14471739, 13)

Merge in GVKEY information for linking with compustat.

gv_cik will come in hand with another merge maybe; but to get to compustat, looks like we just need gvkey - permco.

Some of the keys seem to be used twice. Let's see if they're re-used when a company has gone out of business.

In [174]:
# gv_cik = pd.read_csv("data/cik_gvkey.csv",dtype={'cik':str})[['cik','gvkey']]
gv_permco = pd.read_csv("data/cik_permco.csv",dtype={'cik':str})
# [['gvkey','LPERMCO']].rename(columns = {'LPERMCO': 'PERMCO'}).drop_duplicates()

gv_permco.loc[gv_permco.LINKENDDT == 'E', 'LINKENDDT'] = '20210101'
gv_permco['existsCurrently'] = pd.DatetimeIndex(pd.to_datetime(gv_permco.LINKENDDT.astype(str), format='%Y%m%d')).year

gv_permco = gv_permco[gv_permco.existsCurrently > 2010]
print(gv_permco.shape)
print(gv_permco.head())

gv_permco = gv_permco[['gvkey','LPERMCO','conm']].rename(columns = {'LPERMCO': 'PERMCO'}).drop_duplicates()

(8681, 11)
    gvkey                          conm         cik LINKPRIM LIID LINKTYPE  \
4    1004                      AAR CORP  0000001750        P   01       LU   
48   1045   AMERICAN AIRLINES GROUP INC  0000006201        P   04       LC   
49   1045   AMERICAN AIRLINES GROUP INC  0000006201        P   01       LC   
54   1050       CECO ENVIRONMENTAL CORP  0000003197        P   01       LC   
66   1062  ASA GOLD AND PRECIOUS METALS  0001230869        P   01       LU   

    LPERMNO  LPERMCO    LINKDT LINKENDDT  existsCurrently  
4     54594    20000  19720424  20210101             2021  
48    21020    20010  20131209  20210101             2021  
49    21020    20010  19620131  20120104             2012  
54    11499      176  19801128  20210101             2021  
66    26649    20015  19650129  20210101             2021  


In [164]:
for company in gv_permco[gv_permco.PERMCO.duplicated()].conm:
    print(company)

TYCO INTERNATIONAL PLC
TGC INDUSTRIES INC
GCI LIBERTY INC
TCF FINANCIAL CORP
BAY BANCORP INC
DOCUMENT SECURITY SYS INC
SPHERIX INC
STARWOOD WAYPOINT RES TR
ARC WIRELESS SOLUTIONS INC
GLORI ENERGY INC
SLM CORP
TOBIRA THERAPEUTICS INC
YUMA ENERGY INC -OLD
PANGAEA LOGISTICS SOLUTIONS
EARTHSTONE ENERGY INC
BLUE BIRD CORP
ZAIS GROUP HLDGS INC
GP INVESTMENTS ACQUISITION
LINDBLAD EXPEDITIONS HLDGS
BMC STOCK HOLDINGS INC
PULMATRIX INC
HENNESSY CAPITL ACQ CORP II
AKARI THERAPEUTICS PLC -ADR
PLX PHARMA INC
WINS FINANCE HOLDINGS INC
HERC HOLDINGS INC
ABILITY INC
CENTRIC BRANDS INC
EIGER BIOPHARMACEUTICALS INC
LIBERTY MEDIA BRAVES GROUP
ROAN HOLDINGS GROUP CO LTD
CARDCONNECT CORP
ARCONIC INC
STONE ENERGY CORP
YUMA ENERGY INC
KALVISTA PHARMACEUTICALS
PRIORITY TECHNLGY HLDNGS INC
MICROBOT MEDICAL INC
PMC COMMERCIAL TRUST-OLD
INSPIRED ENTERTAINMENT
WESTROCK CO
TELLURIAN INC
MIRAGEN THERAPEUTICS INC
BMB MUNAI INC -OLD
MEDIA GENERAL INC
ROSEHILL RESOURCES INC
NOVUS THERAPEUTICS INC
CONSTELLATION ALPHA 

Drop the duplicated entries. We may want to revisit this later on.

In [175]:
gv_permco = gv_permco[~gv_permco.PERMCO.isin(list(gv_permco.PERMCO[gv_permco.PERMCO.duplicated()]))]

Now just merge the raw return information with this company information so we can link everything.

In [176]:
rawMergeable = pd.merge(raw,gv_permco,on='PERMCO',how = 'left')
rawMergeable.shape

(14471739, 15)

In [148]:
raw.head()

,PERMNO,date,EXCHCD,SICCD,TICKER,COMNAM,NAICS,PRIMEXCH,PERMCO,PRC,VOL,RET,RETX
5031,14593,20000103,3.0,3573.0,AAPL,APPLE COMPUTER INC,NaN,Q,7,111.9375,4833736,0.088754,0.088754
5032,14593,20000104,3.0,3573.0,AAPL,APPLE COMPUTER INC,NaN,Q,7,102.5000,4646120,-0.084310,-0.084310
5033,14593,20000105,3.0,3573.0,AAPL,APPLE COMPUTER INC,NaN,Q,7,104.0000,7060334,0.014634,0.014634
5034,14593,20000106,3.0,3573.0,AAPL,APPLE COMPUTER INC,NaN,Q,7,95.0000,6917851,-0.086538,-0.086538
5035,14593,20000107,3.0,3573.0,AAPL,APPLE COMPUTER INC,NaN,Q,7,99.5000,4180715,0.047368,0.047368


In [180]:
rawMergeable = rawMergeable.drop(['RETX'],axis = 1)
rawMergeable['date'] = pd.to_datetime(rawMergeable['date'].astype(str), format='%Y%m%d')
rawMergeable['year'] = pd.DatetimeIndex(rawMergeable['date']).year
rawMergeable.head()

,PERMNO,date,EXCHCD,SICCD,TICKER,COMNAM,NAICS,PRIMEXCH,PERMCO,PRC,VOL,RET,gvkey,conm,year
0,14593,2000-01-03,3.0,3573.0,AAPL,APPLE COMPUTER INC,NaN,Q,7,111.9375,4833736,0.088754,1690.0,APPLE INC,2000
1,14593,2000-01-04,3.0,3573.0,AAPL,APPLE COMPUTER INC,NaN,Q,7,102.5000,4646120,-0.084310,1690.0,APPLE INC,2000
2,14593,2000-01-05,3.0,3573.0,AAPL,APPLE COMPUTER INC,NaN,Q,7,104.0000,7060334,0.014634,1690.0,APPLE INC,2000
3,14593,2000-01-06,3.0,3573.0,AAPL,APPLE COMPUTER INC,NaN,Q,7,95.0000,6917851,-0.086538,1690.0,APPLE INC,2000
4,14593,2000-01-07,3.0,3573.0,AAPL,APPLE COMPUTER INC,NaN,Q,7,99.5000,4180715,0.047368,1690.0,APPLE INC,2000


In [181]:
outfile =  'data/stockReturns.pkl'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(rawMergeable, pickle_file)

----------------------------------------------

In [64]:
crsp2010s = crsp[['date','year','PERMNO','COMNAM','PRC']]

(24600859, 14)

In [65]:
sum(crsp.RET == crsp.RETX)

24436450

KeyboardInterrupt: 